In [ ]:
def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        if self.search_type == "similarity":
            docs = self.vectorstore.similarity_search(query, **self.search_kwargs)
        elif self.search_type == "similarity_score_threshold":
            docs_and_similarities = (
                self.vectorstore.similarity_search_with_relevance_scores(
                    query, **self.search_kwargs
                )
            )
            docs = [doc for doc, _ in docs_and_similarities]
        elif self.search_type == "mmr":
            docs = self.vectorstore.max_marginal_relevance_search(
                query, **self.search_kwargs
            )
        else:
            raise ValueError(f"search_type of {self.search_type} not allowed.")
        return docs


In [ ]:
texts = [doc.page_content for doc in documents]
        metadatas = [doc.metadata for doc in documents]
        return cls.from_texts(
            texts=texts,
            embedding=embedding,
            metadatas=metadatas,
            ids=ids,
            collection_name=collection_name,
            persist_directory=persist_directory,
            client_settings=client_settings,
            client=client,
            collection_metadata=collection_metadata,
            **kwargs,
        )


In [ ]:
 chroma_collection = cls(
            collection_name=collection_name,
            embedding_function=embedding,
            persist_directory=persist_directory,
            client_settings=?,
            client=client,
            collection_metadata=collection_metadata,
            **kwargs,
        )
        if ids is None:
            ids = [str(uuid.uuid1()) for _ in texts]
        if hasattr(
            chroma_collection._client, "max_batch_size"
        ):  # for Chroma 0.4.10 and above
            from chromadb.utils.batch_utils import create_batches

            for batch in create_batches(
                api=chroma_collection._client,
                ids=ids,
                metadatas=metadatas,
                documents=texts,
            ):
                chroma_collection.add_texts(
                    texts=batch[3] if batch[3] else [],
                    metadatas=batch[2] if batch[2] else None,
                    ids=batch[0],
                )
        else:
            chroma_collection.add_texts(texts=texts, metadatas=metadatas, ids=ids)
        return chroma_collection


In [ ]:
def __init__(
        self,
        collection_name: str = _LANGCHAIN_DEFAULT_COLLECTION_NAME,
        embedding_function: Optional[Embeddings] = None,
        persist_directory: Optional[str] = None,
        client_settings: Optional[chromadb.config.Settings] = None,
        collection_metadata: Optional[Dict] = None,
        client: Optional[chromadb.Client] = None,
        relevance_score_fn: Optional[Callable[[float], float]] = None,
    ) -> None:
        """Initialize with a Chroma client."""
        try:
            import chromadb
            import chromadb.config
        except ImportError:
            raise ImportError(
                "Could not import chromadb python package. "
                "Please install it with `pip install chromadb`."
            )

        if client is not None:
            self._client_settings = client_settings
            self._client = client
            self._persist_directory = persist_directory
        else:
            if client_settings:
                # If client_settings is provided with persist_directory specified,
                # then it is "in-memory and persisting to disk" mode.
                client_settings.persist_directory = (
                    persist_directory or client_settings.persist_directory
                )
                if client_settings.persist_directory is not None:
                    # Maintain backwards compatibility with chromadb < 0.4.0
                    major, minor, _ = chromadb.__version__.split(".")
                    if int(major) == 0 and int(minor) < 4:
                        client_settings.chroma_db_impl = "duckdb+parquet"

                _client_settings = client_settings
            elif persist_directory:
                # Maintain backwards compatibility with chromadb < 0.4.0
                major, minor, _ = chromadb.__version__.split(".")
                if int(major) == 0 and int(minor) < 4:
                    _client_settings = chromadb.config.Settings(
                        chroma_db_impl="duckdb+parquet",
                    )
                else:
                    _client_settings = chromadb.config.Settings(is_persistent=True)
                _client_settings.persist_directory = persist_directory
            else:
                _client_settings = chromadb.config.Settings()
            self._client_settings = _client_settings
            self._client = chromadb.Client(_client_settings)
            self._persist_directory = (
                _client_settings.persist_directory or persist_directory
            )

        self._embedding_function = embedding_function
        self._collection = self._client.get_or_create_collection(
            name=collection_name,
            embedding_function=None,
            metadata=collection_metadata,
        )
        self.override_relevance_score_fn = relevance_score_fn


In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./emails.txt", encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
for i, text in enumerate(texts):
    text.metadata["source"] = f"{i}-pl"
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

# Start

In [46]:
import chromadb
import chromadb.config

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

import openai

from pprint import pprint

In [23]:
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
collection_name = 'test-col'

In [ ]:
client_settings = chromadb.config.Settings()
pprint(client_settings.__dict__)

In [7]:
client = chromadb.Client(client_settings)
client

In [13]:
collection = client.get_or_create_collection(name=collection_name, embedding_function=None)
collection

Collection(name=test-col)

In [18]:
loader = TextLoader("./emails.txt", encoding="utf-8")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

for i, text in enumerate(texts):
    text.metadata["source"] = f"{i}-pl"
    
texts

[Document(page_content='Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ\u2028Date: Thu, 30 Nov 2023 15:50:44 +0400\nFrom: Ramees Khan | AREC <ramees@arecuae.com>\nTo: info@famaga.ae Dear Sir/Ma’am,\nWe’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.\nPlease quote your best discounted prices and availability.\nI have attached a picture below for your reference.\u2028\u2028Thanking you.\nBest Regards,\nRAMEES KHAN\nGeneral Manager\nM: +971-52-6927466 | E: ramees@arecuae.com\nAL RAMIZ ELECT. CONT. LLC\nA: Industrial Area 17, Sharjah - United Arab Emirates\nA: P.O Box: 96286, Sharjah\nT: +971-6-5356891\nE: info@arecuae.com | W: www.arecuae.com\n\u2028Transformers/Maintenance, Erection & Testing | Rewinding – Transformers, Motors & Generators | Cables – Laying, Termination, Testing – HT & LT | Distribution Boards – MDB, SMDB, DB | Generator Alternator – Repair & Maintenance, Bus Bar Modification & Testing | MV, 

In [21]:
import uuid

ids = [str(uuid.uuid1()) for _ in texts]
ids

['44c64bd0-acd7-11ee-8c92-898860aabb33']

In [24]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [25]:
import tiktoken

In [35]:
tokens = []
indices = []
model_name = 'text-embedding-ada-002'
chunk_size = 1000
embedding_ctx_length = 8191

encoding = tiktoken.encoding_for_model(model_name)

    


In [30]:
texts[0]

Document(page_content='Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ\u2028Date: Thu, 30 Nov 2023 15:50:44 +0400\nFrom: Ramees Khan | AREC <ramees@arecuae.com>\nTo: info@famaga.ae Dear Sir/Ma’am,\nWe’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.\nPlease quote your best discounted prices and availability.\nI have attached a picture below for your reference.\u2028\u2028Thanking you.\nBest Regards,\nRAMEES KHAN\nGeneral Manager\nM: +971-52-6927466 | E: ramees@arecuae.com\nAL RAMIZ ELECT. CONT. LLC\nA: Industrial Area 17, Sharjah - United Arab Emirates\nA: P.O Box: 96286, Sharjah\nT: +971-6-5356891\nE: info@arecuae.com | W: www.arecuae.com\n\u2028Transformers/Maintenance, Erection & Testing | Rewinding – Transformers, Motors & Generators | Cables – Laying, Termination, Testing – HT & LT | Distribution Boards – MDB, SMDB, DB | Generator Alternator – Repair & Maintenance, Bus Bar Modification & Testing | MV, L

In [40]:
token = encoding.encode(text=texts[0].page_content)

len(token)

# len(texts[0].page_content)



395

In [44]:
indicies = []
tokens = []

for j in range(0, len(token), embedding_ctx_length):
    tokens.append(token[j : j + embedding_ctx_length])
    indices.append(i)
        
list(range(0, 20000, embedding_ctx_length))

[0, 8191, 16382]

In [41]:
range??

Init signature: range(self, /, *args, **kwargs)
Docstring:     
range(stop) -> range object
range(start, stop[, step]) -> range object

Return an object that produces a sequence of integers from start (inclusive)
to stop (exclusive) by step.  range(i, j) produces i, i+1, i+2, ..., j-1.
start defaults to 0, and stop is omitted!  range(4) produces 0, 1, 2, 3.
These are exactly the valid indices for a list of 4 elements.
When step is given, it specifies the increment (or decrement).
Type:           type
Subclasses:     

In [ ]:
response = embeddings.client.create(**kwargs)

if any(len(d["embedding"]) == 1 for d in response["data"]) and not skip_empty:
        import openai

        raise openai.error.APIError("OpenAI API returned an empty embedding")


In [47]:
embedding_client = openai.OpenAI().embeddings

embedding_client

In [ ]:
for i, text in enumerate(texts):
    token = encoding.encode(text=text)

    for j in range(0, len(token), self.embedding_ctx_length):
        tokens.append(token[j : j + self.embedding_ctx_length])
        indices.append(i)

batched_embeddings: List[List[float]] = []

for i in _iter:
    response = embedding_client.create(input=tokens[i : i + _chunk_size])
    if any(len(d["embedding"]) == 1 for d in response["data"]) and not skip_empty:
        raise openai.error.APIError("OpenAI API returned an empty embedding")

    if not isinstance(response, dict):
        response = response.dict()
    batched_embeddings.extend(r["embedding"] for r in response["data"])

results: List[List[List[float]]] = [[] for _ in range(len(texts))]
num_tokens_in_batch: List[List[int]] = [[] for _ in range(len(texts))]
for i in range(len(indices)):
    if self.skip_empty and len(batched_embeddings[i]) == 1:
        continue
    results[indices[i]].append(batched_embeddings[i])
    num_tokens_in_batch[indices[i]].append(len(tokens[i]))

embeddings: List[List[float]] = [[] for _ in range(len(texts))]
for i in range(len(texts)):
    _result = results[i]
    if len(_result) == 0:
        average_embedded = embed_with_retry(
                    self,
                    input="",
                    **self._invocation_params,
                )
        if not isinstance(average_embedded, dict):
            average_embedded = average_embedded.dict()
        average = average_embedded["data"][0]["embedding"]
    else:
        average = np.average(_result, axis=0, weights=num_tokens_in_batch[i])
    embeddings[i] = (average / np.linalg.norm(average)).tolist()



In [28]:
encoding.encode??

Signature:
encoding.encode(
    text: 'str',
    *,
    allowed_special: "Union[Literal['all'], AbstractSet[str]]" = set(),
    disallowed_special: "Union[Literal['all'], Collection[str]]" = 'all',
) -> 'list[int]'
Source:   
    def encode(
        self,
        text: str,
        *,
        allowed_special: Union[Literal["all"], AbstractSet[str]] = set(),  # noqa: B006
        disallowed_special: Union[Literal["all"], Collection[str]] = "all",
    ) -> list[int]:
        """Encodes a string into tokens.

        Special tokens are artificial tokens used to unlock capabilities from a model,
        such as fill-in-the-middle. So we want to be careful about accidentally encoding special
        tokens, since they can be used to trick a model into doing something we don't want it to do.

        Hence, by default, encode will raise an error if it encounters text that corresponds
        to a special token. This can be controlled on a per-token level using the `allowed_special`
        a